In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from functools import reduce
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)
def percentchange(x, y):
    try:
        return ((x - y)*100/y)
    except ZeroDivisionError:
        return 0
def realchange(x, y):
    return x-y
def percent(x, y):
    return (x/y)*100

# LAUS  
This outlines how to retrieve labor force statistics from the Local Area Unemployment Statistics via JobsEQ.  

Beginning at data explorer, select monthly data 

In [397]:
data = pd.read_csv('../../Data Downloads/JobsEQLAUS_Monthly.csv')

In [398]:
data = data.set_index('Time').transpose()

In [399]:
data = data.rename(columns = {'Time': 'Datapoint'}).reset_index(drop = False)

In [400]:
data['index'] = data['index'].str.strip()
data['Datapoint'] = data['Datapoint'].str.strip()

In [401]:
boop = data['index'].str.split(pat = ".", expand = True)
data['NAME'] = boop[0]

In [402]:
data = data.drop(columns = 'index').set_index('NAME').transpose()

In [403]:
data = data.rename(columns = {'USA': 'US', 'Tennessee (47)': 'Tennessee', 'Cheatham County, Tennessee (47021)': 'Cheatham County', 
                              'Davidson County, Tennessee (47037)': 'Davidson County', 'Dickson County, Tennessee (47043)': 'Dickson County', 
                              'Houston County, Tennessee (47083)': 'Houston County', 'Humphreys County, Tennessee (47085)': 'Humphreys County', 
                              'Maury County, Tennessee (47119)': 'Maury County', 'Montgomery County, Tennessee (47125)': 'Montgomery County', 
                              'Robertson County, Tennessee (47147)': 'Robertson County', 'Rutherford County, Tennessee (47149)': 'Rutherford County', 
                              'Stewart County, Tennessee (47161)': 'Stewart County', 'Sumner County, Tennessee (47165)': 'Sumner County', 
                              'Trousdale County, Tennessee (47169)': 'Trousdale County', 'Williamson County, Tennessee (47187)': 'Williamson County', 
                              'Wilson County, Tennessee (47189)': 'Wilson County'
                             }).transpose()

In [404]:
data = data.reset_index(drop = False)

In [405]:
cols = data.columns

In [406]:
data = data.melt(var_name = 'Time', id_vars = ['NAME', 'Datapoint'], value_vars = cols, value_name = 'Value')
data.head()

,NAME,Datapoint,Time,Value
0,US,Labor Force (Seas Adj),Jan-00,143568555
1,US,Employed (Seas Adj),Jan-00,137572913
2,US,Unemployed (Seas Adj),Jan-00,6026406
3,US,Unemployment Rate (Seas Adj),Jan-00,0.041
4,Tennessee,Labor Force (Seas Adj),Jan-00,2850230


In [407]:
data = data.set_index('Datapoint').transpose()
data = data.rename(columns = {'Labor Force (Seas Adj)': 'Labor Force', 'Employed (Seas Adj)': 'Employed', 
                              'Unemployed (Seas Adj)': 'Unemployed', 'Unemployment Rate (Seas Adj)': 'Unemployment Rate'})
#data['Unemployment Rate'] = data['Unemployment Rate'] * 100
data = data.transpose().reset_index(drop = False)

In [408]:
data.head()

,Datapoint,NAME,Time,Value
0,Labor Force,US,Jan-00,143568555
1,Employed,US,Jan-00,137572913
2,Unemployed,US,Jan-00,6026406
3,Unemployment Rate,US,Jan-00,0.041
4,Labor Force,Tennessee,Jan-00,2850230


In [409]:
data['Value'] = data['Value'].astype(float)

In [410]:
data['New Value'] = data['Value']

In [411]:
data.loc[data['Datapoint'] == 'Unemployment Rate', 'New Value'] = data['Value'] * 100

In [412]:
#data['month'].value_counts()

In [413]:
data = data.drop(columns = 'Value').rename(columns = {'New Value': 'Value'})

In [414]:
weirds = ['Jan-00', 'Feb-00', 'Mar-00', 'Apr-00', 'May-00', 'Jun-00', 'Jul-00', 'Aug-00', 'Sep-00', 'Oct-00', 'Nov-00', 'Dec-00']

In [415]:
twothousand = data.loc[data['Time'].isin(weirds)]

In [416]:
twothousand.head()

,Datapoint,NAME,Time,Value
0,Labor Force,US,Jan-00,143568555.0
1,Employed,US,Jan-00,137572913.0
2,Unemployed,US,Jan-00,6026406.0
3,Unemployment Rate,US,Jan-00,4.1
4,Labor Force,Tennessee,Jan-00,2850230.0


In [417]:
data = data.loc[~data['Time'].isin(weirds)]

In [418]:
data.head()

,Datapoint,NAME,Time,Value
912,Labor Force,US,1-Jan,144958564.0
913,Employed,US,1-Jan,138798222.0
914,Unemployed,US,1-Jan,6183586.0
915,Unemployment Rate,US,1-Jan,4.2
916,Labor Force,Tennessee,1-Jan,2860282.0


In [419]:
boop = data['Time'].str.split(pat = "-", expand = True)
data['month'] = boop[1]
data['year'] = boop[0]

In [420]:
boop = twothousand['Time'].str.split(pat = "-", expand = True)
twothousand['month'] = boop[0]
twothousand['year'] = boop[1]

In [421]:
df = data.append(twothousand)
df.tail()

C:\Users\jmccall\AppData\Local\Temp\ipykernel_15296\2264709456.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = data.append(twothousand)


,Datapoint,NAME,Time,Value,month,year
907,Unemployment Rate,Williamson County,Dec-00,2.5,Dec,00
908,Labor Force,Wilson County,Dec-00,50170.0,Dec,00
909,Employed,Wilson County,Dec-00,48460.0,Dec,00
910,Unemployed,Wilson County,Dec-00,1688.0,Dec,00
911,Unemployment Rate,Wilson County,Dec-00,3.3,Dec,00


In [422]:
data = df

In [423]:
data['year'] = data['year'].astype(str)
twothousand['year'] = twothousand['year'].astype(str)

In [424]:
#data = data.set_index('Value')

In [425]:
data = data.set_index('month').transpose()
data = data.rename(columns = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 
                              'May': 5, 'Jun': 6, 'Jul': 7, 'Aug': 8, 
                              'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12})

In [426]:
data = data.transpose().reset_index(drop = False)
data = data.set_index('year').transpose()

In [428]:
#data.head()

In [429]:
data = data.rename(columns = {'00': 2000, '1': 2001, '2': 2002, '3': 2003, '4': 2004, '5': 2005, 
                              '6': 2006, '7': 2007, '8': 2008, '9': 2009, '10': 2010, '11': 2011, 
                              '12': 2012, '13': 2013, '14': 2014, '15': 2015, '16': 2016, '17': 2017, 
                              '18': 2018, '19': 2019, '20': 2020, '21': 2021, '22': 2022})

In [430]:
data = data.transpose().reset_index(drop = False)

In [431]:
data['day'] = 1

In [432]:
data['year'].value_counts()

2001    912
2013    912
2022    912
2021    912
2020    912
2019    912
2018    912
2017    912
2016    912
2015    912
2014    912
2012    912
2002    912
2011    912
2010    912
2009    912
2008    912
2007    912
2006    912
2005    912
2004    912
2003    912
2000    912
Name: year, dtype: int64

In [433]:
data.head()

,year,month,Datapoint,NAME,Time,Value,day
0,2001,1,Labor Force,US,1-Jan,144958564.0,1
1,2001,1,Employed,US,1-Jan,138798222.0,1
2,2001,1,Unemployed,US,1-Jan,6183586.0,1
3,2001,1,Unemployment Rate,US,1-Jan,4.2,1
4,2001,1,Labor Force,Tennessee,1-Jan,2860282.0,1


In [434]:
data['month'] = data['month'].astype(int)
data['year'] = data['year'].astype(int)
data['day'] = data['day'].astype(int)

In [435]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20976 entries, 0 to 20975
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   year       20976 non-null  int32 
 1   month      20976 non-null  int32 
 2   Datapoint  20976 non-null  object
 3   NAME       20976 non-null  object
 4   Time       20976 non-null  object
 5   Value      20976 non-null  object
 6   day        20976 non-null  int32 
dtypes: int32(3), object(4)
memory usage: 901.4+ KB


In [436]:
data.head()

,year,month,Datapoint,NAME,Time,Value,day
0,2001,1,Labor Force,US,1-Jan,144958564.0,1
1,2001,1,Employed,US,1-Jan,138798222.0,1
2,2001,1,Unemployed,US,1-Jan,6183586.0,1
3,2001,1,Unemployment Rate,US,1-Jan,4.2,1
4,2001,1,Labor Force,Tennessee,1-Jan,2860282.0,1


In [437]:
data['Test'] = datetime.date(2020, 7, 25)

In [438]:
data['TimeStamp'] = pd.to_datetime(data[['year', 'month', 'day']])

In [180]:
import datetime

In [439]:
data.head()

,year,month,Datapoint,NAME,Time,Value,day,Test,TimeStamp
0,2001,1,Labor Force,US,1-Jan,144958564.0,1,2020-07-25,2001-01-01
1,2001,1,Employed,US,1-Jan,138798222.0,1,2020-07-25,2001-01-01
2,2001,1,Unemployed,US,1-Jan,6183586.0,1,2020-07-25,2001-01-01
3,2001,1,Unemployment Rate,US,1-Jan,4.2,1,2020-07-25,2001-01-01
4,2001,1,Labor Force,Tennessee,1-Jan,2860282.0,1,2020-07-25,2001-01-01


In [440]:
data = data.drop(columns = ['month', 'Time', 'day', 'Test'])

In [441]:
data.head()

,year,Datapoint,NAME,Value,TimeStamp
0,2001,Labor Force,US,144958564.0,2001-01-01
1,2001,Employed,US,138798222.0,2001-01-01
2,2001,Unemployed,US,6183586.0,2001-01-01
3,2001,Unemployment Rate,US,4.2,2001-01-01
4,2001,Labor Force,Tennessee,2860282.0,2001-01-01


In [442]:
data.to_csv('../../Outputs/JOBSEQ_LAUSMONTHLY.csv', index = False)